In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline
import seaborn as sn

In [16]:
import geopandas as gpd
import pandas as pd

## Load daily temperature 

In [3]:
years = [year for year in range(1960, 2020)]

In [ ]:
pattern = []
for year in years:
    temp_year = ddf.read_csv(r'/global/cfs/cdirs/m1532/Projects_MVP/geospatial/PRISM_Data/PRISM_daily_county_level/prism_daily_county_level_' + str(year) + '.csv', dtype = {'year': int, 'fips': str}).compute().drop(columns={'Unnamed: 0'})
    pattern.append(temp_year)

In [5]:
temp = pd.concat(pattern)
temp.head()

,fips,date,tMean,tMin,tMax,prec,year
0,01001,1960-01-01,5.417989,2.635663,8.200316,2.279021,1960
1,01001,1960-01-02,7.005736,4.201073,9.810398,10.412733,1960
2,01001,1960-01-03,9.061477,5.235675,12.887278,16.011329,1960
3,01001,1960-01-04,5.334175,-0.744570,11.412921,0.216112,1960
4,01001,1960-01-05,6.495222,3.006032,9.984412,0.274311,1960


In [6]:
temp['year'].unique()

array([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970,
       1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019])

In [7]:
import math

def water_vapor_pressure(temperature):
    e = 0.61094 * math.exp(17.625 * temperature / (temperature + 243.04))
    return e


def apparent_temperature(temperature, water_vapor_pressure):
    A = -1.3 + 0.92 * temperature + 2.2 * water_vapor_pressure
    return A


temperature = 28 
water_vapor_pressure = water_vapor_pressure(temperature) 
apparent_temp = apparent_temperature(temperature, water_vapor_pressure)
print(f"Apparent temperature at {temperature} degrees Celsius and {water_vapor_pressure:.2f} kPa water vapor pressure is {apparent_temp:.2f} °C")

Apparent temperature at 28 degrees Celsius and 3.77 kPa water vapor pressure is 32.76 °C


In [8]:
def findApparentTemp(temperature):
    e = 0.61094 * math.exp(17.625 * temperature / (temperature + 243.04))
    A = -1.3 + 0.92 * temperature + 2.2 * e
    return A

In [9]:
import datetime

In [10]:
def convertTime(time):
    return datetime.strptime(time, '%Y-%m-%d')

In [11]:
def getMonth(date):
    return(date.strftime('%b'))

In [12]:
temp['date'] = pd.to_datetime(temp['date'])
temp['month'] = temp['date'].apply(getMonth)

In [13]:
month_replace = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10,
                'Nov': 11, 'Dec': 12}

In [14]:
temp['month'] = temp['month'].replace(month_replace)
temp.head()

,fips,date,tMean,tMin,tMax,prec,year,month
0,01001,1960-01-01,5.417989,2.635663,8.200316,2.279021,1960,1
1,01001,1960-01-02,7.005736,4.201073,9.810398,10.412733,1960,1
2,01001,1960-01-03,9.061477,5.235675,12.887278,16.011329,1960,1
3,01001,1960-01-04,5.334175,-0.744570,11.412921,0.216112,1960,1
4,01001,1960-01-05,6.495222,3.006032,9.984412,0.274311,1960,1


In [15]:
climate = temp[['year', 'month', 'date', 'fips', 'tMean', 'tMin', 'tMax']]
climate.head()

,year,month,date,fips,tMean,tMin,tMax
0,1960,1,1960-01-01,01001,5.417989,2.635663,8.200316
1,1960,1,1960-01-02,01001,7.005736,4.201073,9.810398
2,1960,1,1960-01-03,01001,9.061477,5.235675,12.887278
3,1960,1,1960-01-04,01001,5.334175,-0.744570,11.412921
4,1960,1,1960-01-05,01001,6.495222,3.006032,9.984412


In [16]:
climate['AT_mean'] = climate['tMean'].apply(findApparentTemp)
climate['AT_min'] = climate['tMin'].apply(findApparentTemp)
climate['AT_max'] = climate['tMax'].apply(findApparentTemp)
climate.head()

,year,month,date,fips,tMean,tMin,tMax,AT_mean,AT_min,AT_max
0,1960,1,1960-01-01,01001,5.417989,2.635663,8.200316,5.658506,2.748637,8.633515
1,1960,1,1960-01-02,01001,7.005736,4.201073,9.810398,7.347604,4.378348,10.388789
2,1960,1,1960-01-03,01001,9.061477,5.235675,12.887278,9.569066,5.465944,13.821140
3,1960,1,1960-01-04,01001,5.334175,-0.744570,11.412921,5.569946,-0.711797,12.162972
4,1960,1,1960-01-05,01001,6.495222,3.006032,9.984412,6.802082,3.132560,10.580093


In [19]:
climate

,year,month,date,fips,tMean,tMin,tMax,AT_mean,AT_min,AT_max,state
0,1960,1,1960-01-01,01001,5.417989,2.635663,8.200316,5.658506,2.748637,8.633515,01
1,1960,1,1960-01-02,01001,7.005736,4.201073,9.810398,7.347604,4.378348,10.388789,01
2,1960,1,1960-01-03,01001,9.061477,5.235675,12.887278,9.569066,5.465944,13.821140,01
3,1960,1,1960-01-04,01001,5.334175,-0.744570,11.412921,5.569946,-0.711797,12.162972,01
4,1960,1,1960-01-05,01001,6.495222,3.006032,9.984412,6.802082,3.132560,10.580093,01
...,...,...,...,...,...,...,...,...,...,...,...
1133320,2019,12,2019-12-27,56045,-3.423038,-9.134636,2.288561,-3.404296,-9.028568,2.389735,56
1133321,2019,12,2019-12-28,56045,-3.415311,-7.042524,0.211902,-3.396585,-6.984792,0.259813,56
1133322,2019,12,2019-12-29,56045,-4.982109,-7.380541,-2.583677,-4.954086,-7.316181,-2.564803,56
1133323,2019,12,2019-12-30,56045,-5.757468,-10.321875,-1.193062,-5.720491,-10.181061,-1.165475,56


In [20]:
climate.to_csv('heatwave_definition2/PRISM_apparent_temp_1960_2019.csv')

In [17]:
temp = ddf.read_csv(r'/global/cfs/cdirs/m1532/Projects_MVP/geospatial/climate_heatwave/heatwave_1960_2020/heatwave_definition2/PRISM_apparent_temp_1960_2019.csv', dtype = {'year': int, 'fips': str, 'state': str}).compute().drop(columns={'Unnamed: 0'})
temp.head()

,year,month,date,fips,tMean,tMin,tMax,AT_mean,AT_min,AT_max,state
0,1960,1,1960-01-01,01001,5.417989,2.635663,8.200316,5.658506,2.748637,8.633515,01
1,1960,1,1960-01-02,01001,7.005736,4.201073,9.810398,7.347604,4.378348,10.388789,01
2,1960,1,1960-01-03,01001,9.061477,5.235675,12.887278,9.569066,5.465944,13.821140,01
3,1960,1,1960-01-04,01001,5.334175,-0.744570,11.412921,5.569946,-0.711797,12.162972,01
4,1960,1,1960-01-05,01001,6.495222,3.006032,9.984412,6.802082,3.132560,10.580093,01


In [5]:
climate = temp[['year', 'month', 'date', 'fips', 'tMean', 'tMin', 'tMax', 'AT_mean', 'AT_min', 'AT_max', 'state']]
climate.head()

,year,month,date,fips,tMean,tMin,tMax,AT_mean,AT_min,AT_max,state
0,1960,1,1960-01-01,01001,5.417989,2.635663,8.200316,5.658506,2.748637,8.633515,01
1,1960,1,1960-01-02,01001,7.005736,4.201073,9.810398,7.347604,4.378348,10.388789,01
2,1960,1,1960-01-03,01001,9.061477,5.235675,12.887278,9.569066,5.465944,13.821140,01
3,1960,1,1960-01-04,01001,5.334175,-0.744570,11.412921,5.569946,-0.711797,12.162972,01
4,1960,1,1960-01-05,01001,6.495222,3.006032,9.984412,6.802082,3.132560,10.580093,01


In [21]:
def findState(fips):
    return fips[:2]

In [22]:
#climate['state'] = climate['fips'].apply(findState)
climate_state = climate.drop('fips', axis = 1)
climate_state = climate_state.groupby(['year', 'month', 'state']).median()
climate_state = climate_state.reset_index()
climate_state.head()

,year,month,state,date,tMean,tMin,tMax,AT_mean,AT_min,AT_max
0,1960,1,01,1960-01-16,6.831200,2.705458,11.719804,7.160835,2.820909,12.506006
1,1960,1,04,1960-01-16,3.918667,-3.015800,10.344219,4.082961,-2.997460,10.976675
2,1960,1,05,1960-01-16,3.676796,-0.538501,8.495897,3.830463,-0.502942,8.953802
3,1960,1,06,1960-01-16,5.267570,0.236821,10.230564,5.499611,0.285203,10.851253
4,1960,1,08,1960-01-16,-6.266660,-13.284135,1.141567,-6.222319,-13.036280,1.209749


In [6]:
climate['AT_mean'].describe()

count    6.804608e+07
mean     1.405631e+01
std      1.204229e+01
min     -3.634401e+01
25%      5.119291e+00
50%      1.478848e+01
75%      2.386305e+01
max      5.131303e+01
Name: AT_mean, dtype: float64

In [7]:
climate['year'].unique()

array([1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970,
       1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981,
       1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992,
       1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003,
       2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014,
       2015, 2016, 2017, 2018, 2019])

In [8]:
climate = climate.sort_values(['fips', 'month', 'year'])
climate

,year,month,date,fips,tMean,tMin,tMax,AT_mean,AT_min,AT_max,state
0,1960,1,1960-01-01,01001,5.417989,2.635663,8.200316,5.658506,2.748637,8.633515,01
1,1960,1,1960-01-02,01001,7.005736,4.201073,9.810398,7.347604,4.378348,10.388789,01
2,1960,1,1960-01-03,01001,9.061477,5.235675,12.887278,9.569066,5.465944,13.821140,01
3,1960,1,1960-01-04,01001,5.334175,-0.744570,11.412921,5.569946,-0.711797,12.162972,01
4,1960,1,1960-01-05,01001,6.495222,3.006032,9.984412,6.802082,3.132560,10.580093,01
...,...,...,...,...,...,...,...,...,...,...,...
438458,2019,12,2019-12-27,56045,-3.423038,-9.134636,2.288561,-3.404296,-9.028568,2.389735,56
438459,2019,12,2019-12-28,56045,-3.415311,-7.042524,0.211902,-3.396585,-6.984792,0.259813,56
438460,2019,12,2019-12-29,56045,-4.982109,-7.380541,-2.583677,-4.954086,-7.316181,-2.564803,56
438461,2019,12,2019-12-30,56045,-5.757468,-10.321875,-1.193062,-5.720491,-10.181061,-1.165475,56


In [9]:
climate['date'] = pd.to_datetime(climate['date'])

In [10]:
# Filter for July and August, from 1981 to 2010
df_jul_aug = climate[(climate['date'].dt.month.isin([7, 8])) & (climate['date'].dt.year.between(1981, 2010))]


In [11]:
df_jul_aug

,year,month,date,fips,tMean,tMin,tMax,AT_mean,AT_min,AT_max,state
120307,1981,7,1981-07-01,01001,25.258767,19.771108,30.746426,29.001928,21.950693,36.714649,01
120308,1981,7,1981-07-02,01001,22.747374,19.327411,26.167336,25.702292,21.404888,30.228882,01
120309,1981,7,1981-07-03,01001,23.346042,19.872791,26.819293,26.477125,22.076246,31.120841,01
120310,1981,7,1981-07-04,01001,24.560164,19.250909,29.869419,28.070840,21.311116,35.430792,01
120311,1981,7,1981-07-05,01001,25.964896,21.254444,30.675348,29.953901,23.800192,36.609815,01
...,...,...,...,...,...,...,...,...,...,...,...
332337,2010,8,2010-08-27,56045,24.817030,14.288773,35.345287,28.411967,15.422117,43.814747,56
332338,2010,8,2010-08-28,56045,23.812539,14.602777,33.022301,27.085878,15.784301,40.147719,56
332339,2010,8,2010-08-29,56045,22.271928,13.423014,31.120841,25.091945,14.430186,37.269204,56
332340,2010,8,2010-08-30,56045,21.234611,12.363376,30.105845,23.775204,13.228960,35.774844,56


In [12]:
percentiles = df_jul_aug.groupby('fips')['AT_min'].quantile(0.85).reset_index()
percentiles.rename(columns={'AT_min': 'threshold_temp'}, inplace=True)

In [13]:
percentiles

,fips,threshold_temp
0,01001,25.319677
1,01003,26.775692
2,01005,24.736532
3,01007,24.910650
4,01009,24.220104
...,...,...
3100,56037,12.578402
3101,56039,7.031365
3102,56041,11.381369
3103,56043,15.771672


In [14]:
# Merge the thresholds back to the original DataFrame
df_climate = climate.merge(percentiles, on='fips')

# Identify exceedances
df_climate['is_exceedance'] = df_climate['AT_min'] > df_climate['threshold_temp']
df_climate.head()

,year,month,date,fips,tMean,tMin,tMax,AT_mean,AT_min,AT_max,state,threshold_temp,is_exceedance
0,1960,1,1960-01-01,01001,5.417989,2.635663,8.200316,5.658506,2.748637,8.633515,01,25.319677,False
1,1960,1,1960-01-02,01001,7.005736,4.201073,9.810398,7.347604,4.378348,10.388789,01,25.319677,False
2,1960,1,1960-01-03,01001,9.061477,5.235675,12.887278,9.569066,5.465944,13.821140,01,25.319677,False
3,1960,1,1960-01-04,01001,5.334175,-0.744570,11.412921,5.569946,-0.711797,12.162972,01,25.319677,False
4,1960,1,1960-01-05,01001,6.495222,3.006032,9.984412,6.802082,3.132560,10.580093,01,25.319677,False


In [ ]:
df_climate['block'] = (df_climate['is_exceedance'] != df_climate['is_exceedance'].shift(1)).cumsum()
df_heatwaves = df_climate.groupby(['fips', 'block']).agg(
    start_date=('date', 'min'),
    end_date=('date', 'max'),
    exceedance_days=('is_exceedance', 'sum')
)

In [ ]:
df_heatwaves = df_heatwaves.reset_index()
df_heatwaves

In [ ]:
# Filter for periods where exceedance days are 2 or more
df_heatwaves = df_heatwaves[df_heatwaves['exceedance_days'] >= 2]
df_heatwaves

In [ ]:
df_heatwaves['year'] = df_heatwaves['start_date'].dt.year
df_heatwaves['month'] = df_heatwaves['start_date'].dt.month
df_heatwaves.head()

In [ ]:
heatwave_days = df_heatwaves.drop(['block', 'start_date', 'end_date'], axis = 1)
heatwave_days = heatwave_days.groupby(['fips', 'year', 'month']).sum()
heatwave_days = heatwave_days.reset_index()
heatwave_days = heatwave_days.rename(columns = {'exceedance_days': 'heatwave_days'})
heatwave_days

In [39]:
heatwave_days.to_csv('heatwave_definition2/heatwave_days_county_level.csv')

In [40]:
df_heatwaves['year'] = df_heatwaves['start_date'].dt.year
df_heatwaves['month'] = df_heatwaves['start_date'].dt.month

yearly_heatwaves = df_heatwaves.groupby(['fips', 'year', 'month']).size().reset_index(name='heatwave_count')
yearly_heatwaves

/tmp/ipykernel_1638401/1649591901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_heatwaves['year'] = df_heatwaves['start_date'].dt.year
/tmp/ipykernel_1638401/1649591901.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_heatwaves['month'] = df_heatwaves['start_date'].dt.month


,fips,year,month,heatwave_count
0,01001,1960,6,1
1,01001,1960,7,2
2,01001,1962,7,1
3,01001,1963,6,1
4,01001,1963,8,1
...,...,...,...,...
288458,56045,2016,7,1
288459,56045,2017,7,4
288460,56045,2017,8,1
288461,56045,2018,7,1


In [41]:
yearly_heatwaves.to_csv('heatwave_definition2/heatwave_count_county_level.csv')

### Adjust the heatwave days

In [3]:
days_heatwave = ddf.read_csv('heatwave_definition2/heatwave_days_county_level_raw.csv', dtype = {'year': int, 'fips': str}).compute().drop(columns={'Unnamed: 0'})
days_heatwave.head()

,fips,year,month,heatwave_days
0,01001,1960,6,2
1,01001,1960,7,5
2,01001,1962,7,2
3,01001,1963,6,2
4,01001,1963,8,2


In [4]:
days_heatwave['heatwave_days'].describe()

count    288463.000000
mean          5.203250
std           3.965798
min           2.000000
25%           2.000000
50%           4.000000
75%           7.000000
max          60.000000
Name: heatwave_days, dtype: float64

In [5]:
heatwave_over = days_heatwave[days_heatwave['heatwave_days'] >= 31]
heatwave_over

,fips,year,month,heatwave_days
7142,05019,2010,8,34
7465,05027,2010,8,33
7962,05039,2010,8,33
8518,05053,2010,8,31
8682,05057,2010,8,32
...,...,...,...,...
254196,48491,2010,8,39
254429,48495,2011,8,32
254523,48497,2010,8,33
254610,48499,2010,8,34


In [6]:
month_31days = [1, 3, 5, 7, 8, 10, 12]
month_30days = [4, 6, 9, 11]
month_28days = [2]

In [7]:
def is_leap_year(year):
    if (year % 4 == 0 and year % 100 != 0) or year % 400 == 0:
        return 1
    else:
        return 0

In [8]:
days_heatwave['is_leap'] = days_heatwave['year'].apply(is_leap_year)
days_heatwave.head()

,fips,year,month,heatwave_days,is_leap
0,01001,1960,6,2,1
1,01001,1960,7,5,1
2,01001,1962,7,2,0
3,01001,1963,6,2,0
4,01001,1963,8,2,0


In [9]:
def adjust_days(is_leap, month, days):
    if is_leap == 1 and month == 2 and days >=29:
        return 29
    elif is_leap == 0 and month == 2 and days >=28:
        return 28
    elif (month in month_31days) and days >= 31:
        return 31
    elif (month in month_30days) and days >= 30:
        return 30
    else:
        return days

In [10]:
days_heatwave['adjusted_days'] = days_heatwave.apply(lambda x: adjust_days(x['is_leap'], x['month'], x['heatwave_days']), axis=1)
days_heatwave.head()

,fips,year,month,heatwave_days,is_leap,adjusted_days
0,01001,1960,6,2,1,2
1,01001,1960,7,5,1,5
2,01001,1962,7,2,0,2
3,01001,1963,6,2,0,2
4,01001,1963,8,2,0,2


In [11]:
days_heatwave['adjusted_days'].describe()

count    288463.000000
mean          5.199825
std           3.938827
min           2.000000
25%           2.000000
50%           4.000000
75%           7.000000
max          31.000000
Name: adjusted_days, dtype: float64

In [12]:
days_heatwave = days_heatwave.drop(['heatwave_days', 'is_leap'], axis = 1)
days_heatwave = days_heatwave.rename(columns = {'adjusted_days': 'heatwave_days'})
days_heatwave.head()

,fips,year,month,heatwave_days
0,01001,1960,6,2
1,01001,1960,7,5
2,01001,1962,7,2
3,01001,1963,6,2
4,01001,1963,8,2


In [13]:
days_heatwave['heatwave_days'].describe()

count    288463.000000
mean          5.199825
std           3.938827
min           2.000000
25%           2.000000
50%           4.000000
75%           7.000000
max          31.000000
Name: heatwave_days, dtype: float64

In [14]:
days_heatwave.to_csv('heatwave_definition2/heatwave_days_county_level.csv')